<a href="https://colab.research.google.com/github/golesuman/Deep-learning-withTensorflowKeras/blob/main/trainimage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import tensorflow as tf
# from tensorflow.keras import layers,models,
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models
import os
import cv2 as cv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!unzip /content/drive/MyDrive/Data.zip

Archive:  /content/drive/MyDrive/Data.zip
replace Data/train/Accident/Accident54.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [36]:
src_train_path = '/content/Data/train'
src_test_path = '/content/Data/test'
# src_test_valid = '/content/Data/'
train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)

In [37]:
batch_size = 8
train_generator = train_datagen.flow_from_directory(
    directory= src_train_path,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)
valid_generator = train_datagen.flow_from_directory(
    directory=src_train_path,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    directory=src_test_path,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)



Found 245 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 48 images belonging to 2 classes.


In [40]:
def prepare_model():
    layers.Conv2D(64,(3,3),input_shape=(100,100,1),activation='relu'),
    layers.MaxPool2D(3,3),
    
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D(3,3),
    
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D(3,3),
    
    layers.Flatten(),
    layers.Dense(1000,activation='relu'),
    layers.Dense(2,activation='softmax')
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
    return model

model = prepare_model()

In [41]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 76832)             0         
                                                                 
 dense_12 (Dense)            (None, 16)                1229328   
                                                                 
 dense_13 (Dense)            (None, 2)                 34        
                                                                 
Total params: 1,230,258
Trainable params: 1,230,258
Non-trainable params: 0
____________________________________________

In [42]:
model = prepare_model()
model.fit_generator(train_generator,
                    validation_data = train_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=20)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


30/30 [==============================] - 3s 99ms/step - loss: 0.4450 - accuracy: 0.8101 - val_loss: 0.4669 - val_accuracy: 0.8750
Epoch 2/20
30/30 [==============================] - 3s 94ms/step - loss: 0.4882 - accuracy: 0.8228 - val_loss: 0.4249 - val_accuracy: 0.8571
Epoch 3/20
30/30 [==============================] - 3s 92ms/step - loss: 0.4156 - accuracy: 0.8523 - val_loss: 0.4386 - val_accuracy: 0.8036
Epoch 4/20
30/30 [==============================] - 3s 96ms/step - loss: 0.5088 - accuracy: 0.7833 - val_loss: 0.4231 - val_accuracy: 0.8571
Epoch 5/20
30/30 [==============================] - 3s 94ms/step - loss: 0.4609 - accuracy: 0.8354 - val_loss: 0.3921 - val_accuracy: 0.9107
Epoch 6/20
30/30 [==============================] - 3s 92ms/step - loss: 0.4242 - accuracy: 0.8417 - val_loss: 0.3856 - val_accuracy: 0.8571
Epoch 7/20
30/30 [==============================] - 3s 101ms/step - loss: 0.3964 - accuracy: 0.8945 - val_loss: 0.4011 - val_accuracy: 0.8750
Epoch 8/20
30/30 [=====

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [43]:
score = model.evaluate_generator(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


Test loss: 1.3077470064163208
Test accuracy: 0.75


In [45]:
predict=model.predict_generator(test_generator)
# predict the class label
y_classes = predict.argmax(axis=-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [46]:
print(y_classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1]


In [47]:
y_classes.size

48

In [ ]:
count = 0
for i in range(30):
  if y_classes[i] == 0:
    count +=1

print(count)
    

In [53]:
model.save('accidentornot.h5')

In [63]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
class_names = ['Accident', 'NotAccident'] # fill the rest

model = load_model('accidentornot.h5')

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


def preprocess(image_path):
  img = cv2.imread('/content/noacci.jpeg')
  img = cv2.resize(img,(100,100))
  img = np.reshape(img,[1,100,100,3])
  return img

def give_classes(img):
  classes = np.argmax(model.predict(img), axis = -1)
  names = [class_names[i] for i in classes]
  return classes, names



In [66]:
new_img = preprocess('/content/noacci.jpeg')
classid, name = give_classes(new_img)
print(classid, name)

[1] ['NotAccident']
